# Pop2Piano - Inference (Use Ready Model) 🎹

This notebook is for generating piano covers using the pre-trained Pop2Piano model.

## Key Steps:
1. **Setup Environment:** Install dependencies and fix compatibility issues.
2. **Clone Repository:** Get the code.
3. **Load Model:** Download/Load the pre-trained model.
4. **Generate:** Upload audio and create piano MIDI.

In [ ]:
# @title 1. Install Dependencies 📦
import os
import sys
import subprocess

print("Installing dependencies... ⏳")

# Install Python packages (stable versions only)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "transformers", "essentia", "librosa", "resampy", "pretty_midi", "midi2audio", "scipy", "yt-dlp", "imageio-ffmpeg", "omegaconf", "joblib", "tqdm"])

# Install System packages
os.system("sudo apt-get update -q")
os.system("sudo apt-get install -y -q fluidsynth fluid-soundfont-gm ffmpeg")

print("Done! ✅")

In [ ]:
# @title 2. Apply Magic Fixes (DISABLED) 🪄
# We DISABLED the mocking because Inference requires real Essentia to analyze audio rhythm.
# Only enable this if you are debugging environment issues and NOT running inference.

print("Skipping Mocking... We need the real Essentia library! 🎵")

# import sys
# from unittest.mock import MagicMock
# from importlib.machinery import ModuleSpec

# # Fix for missing essentia
# mock_essentia = MagicMock()
# # Important fix for Python 3.12+ (spec must be set)
# mock_essentia.__spec__ = ModuleSpec(name='essentia', loader=None)

# sys.modules["essentia"] = mock_essentia
# sys.modules["essentia.standard"] = MagicMock()

# print("Essentia mocked successfully! 🧙‍♂️")

In [ ]:
# @title 3. Clone Repository 🐙
import os

repo_url = "https://github.com/kareemkamal10/pop2piano.git"
repo_name = "pop2piano"

# Work in /content (Colab's default, no Drive needed)
%cd /content

if os.path.exists(repo_name):
    print("Repository already cloned. Pulling latest changes...")
    %cd {repo_name}
    !git pull
else:
    print(f"Cloning {repo_url}...")
    !git clone {repo_url}
    %cd {repo_name}

print(f"Current working directory: {os.getcwd()}")

In [ ]:
# @title 4. Load Pre-trained Model & Generate 🎹
from transformers import Pop2PianoForConditionalGeneration, Pop2PianoProcessor
from google.colab import files
import librosa
import torch
import soundfile as sf

# 1. Load Model
print("⏳ Loading Model...")
model = Pop2PianoForConditionalGeneration.from_pretrained("sweetcocoa/pop2piano")
processor = Pop2PianoProcessor.from_pretrained("sweetcocoa/pop2piano")
print("✅ Model Loaded!")

# 2. Upload Audio File
print("\n📤 Please upload your audio file (MP3/WAV)...")
uploaded = files.upload()
audio_filename = next(iter(uploaded))
print(f"🎵 Processing: {audio_filename}")

# 3. Process & Generate
audio, rate = librosa.load(audio_filename, sr=None) 
inputs = processor(audio=audio, sampling_rate=rate, return_tensors="pt")

print("🎹 Generating MIDI... (This may take a minute)")

# FIX: Only pass input_features to generate(), keep beatsteps for decoding
outputs = model.generate(
    input_features=inputs["input_features"],
    composer="composer1"
)

# 4. Save & Download
output_midi_name = "output_midi.mid"
# batch_decode needs the feature_extractor_output (which contains beatsteps)
processor.batch_decode(
    token_ids=outputs,
    feature_extractor_output=inputs,
    return_midi=True
)[0].write(output_midi_name)

print(f"✨ Success! Saved to {output_midi_name}")
files.download(output_midi_name)